In [1]:
from gurobipy import *

# Model data
gens,maxGen,Co,a,b,suCo,ruLi,rdLi,ipOut,init=multidict({
'g1':[400.0,100.0,20.0,0.05,300.0,160.0,160.0,0.0,0.0],
'g2':[300.0,200.0,25.0,0.10,400.0,150.0,150.0,0.0,0.0],
'g3':[250.0,300.0,40.0,0.20,500.0,100.0,100.0,0.0,0.0]})

times, demand, reLevel = multidict({
'h1': [260.0, 260.0*0.2],
'h2': [500.0, 500.0*0.2],
'h3': [700.0, 700.0*0.2],
'h4': [300.0, 300.0*0.2]})

# auxiliar list
t2=['h1','h2','h3','h4','h5']

In [2]:
# Create optimization model
m = Model('UC')

## Generation per Unit Variables $\quad P_{Gjt} \quad \forall j, \forall t$

In [3]:
# variable for generations per unit per hour
Pg = {}
for g in gens:
    for t in times:
        # 12 variables
        Pg[g,t] = m.addVar(lb=0.0, ub=maxGen[g], name='Pg'+'[%s,%s]'%(g,t))
        # initial value 0.0
        Pg[g,t].start = 0.0
#update model
m.update()        

## Status on/off variables   $\quad u_{jt} = \{0,1\} \quad \forall j, \forall t$

In [4]:
# variable for generations per unit per hour
u = {}
for g in gens:
    for t in times:
        # 12 variables
        u[g,t] = m.addVar(vtype=GRB.BINARY, name='u'+'[%s,%s]'%(g,t))
        # initial value 0
        u[g,t].start = 0
#update model
m.update()      

## StartUp Cost variables $ \quad C_{jt}^{SU} $

In [5]:
# variable for Startup Costs per hour
su = {}
for g in gens:
    for t in times:
        # 12 variables
        su[g,t] = m.addVar(name='su'+'[%s,%s]'%(g,t))
        # initial value 0
        #su[g,t].start = 0
#update model
m.update()   

## Objective
$ min \sum_{t=1}^T\sum_{j=1}^n C_{jt}(u_{jt},P_{Gjt})+C_{jt}^{SU}$

In [6]:
m.setObjective(sum(u[g,t]*Co[g] +
                   a[g]*Pg[g,t] +
                   0.5*b[g]*Pg[g,t]*Pg[g,t] +
                   su[g,t]
                   for g in gens 
                   for t in times), 
                   GRB.MINIMIZE)
m.update()

## Min-Max Generation Constraint 
$ u_{jt}P_{Gjt}^{min} \le P_{Gjt} \le u_{jt}P_{Gjt}^{max}$

In [7]:
m.addConstrs((Pg[g,t] >= 0.0 
              for g in gens
              for t in times),
              name='pMin')
m.addConstrs((u[g,t]*maxGen[g] >= Pg[g,t]
              for g in gens
              for t in times),
              name='pMax')
m.update()

## Constraint for supply-demand 
$\sum_{j=1}^n P_{Gjt} = P_{Dt} \quad \forall j, \forall t$

In [8]:
m.addConstrs(
    (quicksum(Pg[g,t] for g in gens) == demand[t]
     for t in times), name='meetDemand')
m.update()

## Constraint for RampDown 

In [9]:
for g in gens:
    for t in range(4):
        if t2[t-1] == 'h5':
            m.addConstr(0.0 - Pg[g,t2[t]] <= rdLi[g],
                        name='rampDown'+'[%s,%s]'%(g,t2[t]))
            continue
        m.addConstr(Pg[g,t2[t-1]] - Pg[g,t2[t]] <= rdLi[g], 
                    name='rampDown'+'[%s,%s]'%(g,t2[t]))
m.update()

## Constraint for RampUp

In [10]:
print(m.getConstrs())

[<gurobi.Constr pMin[g3,h2]>, <gurobi.Constr pMin[g3,h3]>, <gurobi.Constr pMin[g3,h1]>, <gurobi.Constr pMin[g3,h4]>, <gurobi.Constr pMin[g2,h2]>, <gurobi.Constr pMin[g2,h3]>, <gurobi.Constr pMin[g2,h1]>, <gurobi.Constr pMin[g2,h4]>, <gurobi.Constr pMin[g1,h2]>, <gurobi.Constr pMin[g1,h3]>, <gurobi.Constr pMin[g1,h1]>, <gurobi.Constr pMin[g1,h4]>, <gurobi.Constr pMax[g3,h2]>, <gurobi.Constr pMax[g3,h3]>, <gurobi.Constr pMax[g3,h1]>, <gurobi.Constr pMax[g3,h4]>, <gurobi.Constr pMax[g2,h2]>, <gurobi.Constr pMax[g2,h3]>, <gurobi.Constr pMax[g2,h1]>, <gurobi.Constr pMax[g2,h4]>, <gurobi.Constr pMax[g1,h2]>, <gurobi.Constr pMax[g1,h3]>, <gurobi.Constr pMax[g1,h1]>, <gurobi.Constr pMax[g1,h4]>, <gurobi.Constr meetDemand[h2]>, <gurobi.Constr meetDemand[h3]>, <gurobi.Constr meetDemand[h1]>, <gurobi.Constr meetDemand[h4]>, <gurobi.Constr rampDown[g3,h1]>, <gurobi.Constr rampDown[g3,h2]>, <gurobi.Constr rampDown[g3,h3]>, <gurobi.Constr rampDown[g3,h4]>, <gurobi.Constr rampDown[g2,h1]>, <gurobi.Co

In [ ]:
#constraints
def generation_r(model,i):
    return (model.Pmin[i] , model.P[i] , model.Pmax[i])
model.GenCon = Constraint(model.G, rule = generation_r)

def demand_r(model):
    return model.D == sum(model.P[i] for i in model.G)
model.DemandCon = Constraint(model.G, rule = demand_r)

def bus_r(model,i):
    if i == 4:
        return(-model.Loads[i] == 
              ((model.BaseP/model.X[i,i-1])*
               (model.Angles[i]-model.Angles[i-1]) + 
               (model.BaseP/model.X[i,i-3])*
               (model.Angles[i]-model.Angles[i-3])))
    elif i == 1:
        return((model.P[i] - model.Loads[i]) == 
              ((model.BaseP/model.X[i,i+1])*
               (model.Angles[i]-model.Angles[i+1]) + 
               (model.BaseP/model.X[i,i+3])*
               (model.Angles[i]-model.Angles[i+3])))    
    else:
        return((model.P[i] - model.Loads[i]) ==     
              ((model.BaseP/model.X[i,i+1])*
               (model.Angles[i]-model.Angles[i+1]) +
               (model.BaseP/model.X[i,i-1])*
               (model.Angles[i]-model.Angles[i-1])))
model.BusCon = Constraint(model.Bus, rule=bus_r)

def flow_r(model,i,j):
    if i == j:
        return Constraint.Skip
    return (model.PFmin , 
            (model.BaseP/model.X[i,j])*(model.Angles[i]-model.Angles[j]) , 
            model.PFmax)
model.FlowCon = Constraint(model.Bus, model.Bus, rule=flow_r)

In [ ]:
#objective
def cost_rule(model):
    return sum(model.a[i]*model.P[i] + 
               0.5*model.b[i]*model.P[i]**2
              for i in model.G)
#default is to minimize        
model.OBJ = Objective(rule=cost_rule) 

In [ ]:
# print(gens)
# print(gen['gen3'].getAttr('UB'))

#print(m.getObjective())
# print(m.getConstrs())
# print(m.getConstrs()[25])
# print(m.getConstrs()[25].getAttr('sense'))
# print(m.getConstrs()[25].getAttr('rhs'))

# print reLevel

# #number of variables
# print(m.numVars)

# print(maxGen['gen2'])
# print(m.numVars)
# print(genPhour)
# print(genPhour['gen3','hour1'].getAttr('UB'))
# print(genPhour['gen3','hour1'].getAttr('start'))
# print(ujt['gen3','hour1'].getAttr('LB'))

#variable for generations per unit
#gen = m.addVars(gens, lb=0.0, ub=maxGen, name='gen')

# m.addConstrs(
# (quicksum(nutritionValues[f,c] * buy[f] for f in foods) == nutrition[c]
# for c in categories), "_")

# for t in range(4):
#     if t2[t-1] == 'h5':
#         print('t ' + t2[t])
#         print('t-1 0')
#         continue
#     print('t ' + t2[t])
#     print('t-1 ' + t2[t-1])